In [8]:
import os
from matplotlib import axis
import scipy.io as sio
import numpy as np
import mne
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
import pandas as pd
from torch.utils.data import TensorDataset
from sklearn.model_selection import KFold, train_test_split
import model as dl  # Ensure this module contains necessary utility functions
import logging
from mne.preprocessing import ICA
import os
import warnings
# 忽略 RuntimeWarning 警告
warnings.filterwarnings("ignore", category=RuntimeWarning)
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [9]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [10]:
def crop_save(file_paths, duration,type,save_base_dir):
    num_nomal =0
    num_mci = 0
    EEG =0
    for i, file in enumerate(file_paths):
        try:
            raw = mne.io.read_raw_edf(file, preload=True, encoding='latin1',verbose='Warning')
            data = raw.get_data()[0:19]
            name = file[14:][:-4]
            if data.shape[1] > duration:
                epochs = data.shape[1] // duration
                data_crop = data[:,0:epochs*duration]
            else:
                continue
            channels = data.shape[0]
            data_new = data_crop.reshape(channels, -1, duration).transpose(1, 0, 2)
            for j in range(epochs):
                part = pd.DataFrame(data_new[j])
                part = part.reset_index(drop=True)
                if(type == 0):
                    dir = save_base_dir+"\\认知正常\\"+name+"\\"
                    ensure_dir(dir)
                    file_path = dir+name+"_"+str(j+1)+'.csv'
                    part.to_csv(file_path)
                else:
                    dir = save_base_dir+"\\认知障碍\\"+name+"\\"
                    ensure_dir(dir)
                    file_path = dir+name+"_"+str(j+1)+'.csv'
                    part.to_csv(file_path)
            if type==0:
                num_nomal+=epochs
            else:
                num_mci+=epochs
            EEG += epochs
            logging.info(f"Processed file {file}: {epochs} epochs")
        except Exception as e:
            logging.error(f"Error processing file {file}: {e}")
            continue
    logging.info(f"Total epochs: {EEG}, Normal: {num_nomal}, "f"MCI: {num_mci}")

In [ ]:

# 忽略 RuntimeWarning 警告
warnings.filterwarnings("ignore", category=RuntimeWarning)
# 定义文件夹路径
base_dir = '糖尿病数据ICA'
normal_dir = os.path.join(base_dir, '认知正常')
impaired_dir = os.path.join(base_dir, '认知障碍')

# 获取所有的文件路径
normal_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir) if f.endswith('.edf')]
impaired_files = [os.path.join(impaired_dir, f) for f in os.listdir(impaired_dir) if f.endswith('.edf')]

save_dir = "糖尿病数据ICA分段"
crop_save(normal_files,duration=1000,type=0,save_base_dir=save_dir)
crop_save(impaired_files,duration=1000,type=1,save_base_dir=save_dir)

In [ ]:
impaired_files